# Interactive Plots

This notebook is used to generate interactive visualizations with the python plotly library for use in the online documentation.

In [1]:
import numpy as np

import plotly
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle

from plot_funcs import (get_results, get_ranks_sizes,
                        get_highest_performing_df, get_cut_off_df,
                        get_across_ranks, get_intra_pipeline_df,
                        get_single_vs_multiple_df, get_results_df,
                        get_single_vs_multiple_df)

/home/sage/anaconda3/envs/bpt/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
results = get_results('../exp/results')

static = {'random': True, 'base': True, 'fs': True, 'ico': True}
ensemble_only = {'stacked': True, 'voted': True}
all_parcels = {**static, **ensemble_only, 'grid': True}

params = dict(results=results, keep_full_name=True, threshold=False, **static)

ts = [True, True, True, True]
fs = [False, False, False, False]

Found: 87390 Incomplete: 0


In [3]:
def init_fig(ylabel='Mean Rank'):
    
    fig = go.Figure()
    fig.layout.xaxis.title = 'Size'
    fig.layout.yaxis.title = ylabel
    
    return fig

def add_base_side_annot(fig, buttons):
    
    # Add dropdown
    fig.update_layout(
        updatemenus=[
            dict(
                type = "buttons",
                direction = "left",
                buttons=buttons,
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.11,
                xanchor="left",
                y=1.1,
                yanchor="top"
            ),
        ]
    )

    # Add annotation
    fig.update_layout(
        annotations=[
            dict(text="Scale:", showarrow=False,
                 x=0, y=1.08, yref="paper", align="left")
        ]
    )

def add_go_base_scatter(fig, name, df, visible, y='Mean_Rank', marker_size=20):
    
    gs = go.Scatter(
            x=df['Size'],
            y=df[y],
            visible=visible,
            mode='markers',
            name=name,
            hovertext=df['full_name'],
            textposition="middle center",
            customdata=np.stack((df['r2'],
                                 df['roc_auc'],
                                 df['rank_label'],
                                 df['size_label']), axis=-1),
            hovertemplate='<b>%{hovertext}</b><br>' +
                          '%{customdata[2]}<br>' +
                          '%{customdata[3]}<br>' +
                          'Mean R2: %{customdata[0]:.4f}<br>' +
                          'Mean ROC AUC: %{customdata[1]:.4f}<br>' +
                          '<extra></extra>')

    gs.marker.size = marker_size
    gs.marker.opacity = .5

    # Add plot
    fig.add_trace(gs)
    
def add_go_by_target_scatter(fig, name, df, visible,
                             palette, y='Mean_Rank',
                             marker_size=20):
    
    gs = go.Scatter(
            x=df['Size'],
            y=df[y],
            visible=visible,
            mode='markers',
            name=name,
            text=df['full_name'],
            customdata=np.stack((df['rank_label'],
                                 df['size_label'],
                                 df['Mean_Score']), axis=-1),
            hovertemplate='<b>%{text}</b><br>' +
                          '<b>' + name + '</b><br>' + 
                          '%{customdata[0]}<br>' +
                          '%{customdata[1]}<br>' +
                          'Mean Raw Metric: %{customdata[2]:.4f}<extra></extra>')

    gs.marker.size = marker_size
    gs.marker.opacity = .25
    gs.marker.color = next(palette)

    # Add plot
    fig.add_trace(gs)
    
def by_pipeline_base(metric_type='Mean_Rank', marker_size=20,
                     xrange=(.5, 4), yrange=(1.2, 2.8),
                     log=True, **parcels):
    
    models = ['LGBM', 'SVM', 'Elastic-Net', 'All']

    fig_dict = {
        "data": [],
        "layout": {},
        "frames": []
    }
    
    # Get ylabel
    if metric_type == 'Mean_Rank':
        ylabel = 'Mean Rank'
        log_raw = False
    elif metric_type == 'r2':
        ylabel = 'Mean R2'
        log_raw = True
    elif metric_type == 'roc_auc':
        ylabel = 'Mean ROC AUC'
        log_raw = True
        
        
    extra = ' (log10)'
    if not log:
        log_raw = False
        extra = ''

    fig_dict["layout"]["xaxis"] = {"range": xrange, "title": f"Size{extra}"}
    fig_dict["layout"]["yaxis"] = {"range": yrange, "title": f"{ylabel}{extra}"}
    fig_dict["layout"]["hovermode"] = "closest"

    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Pipeline:",
            "visible": True,
            "xanchor": "right"
        },
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }

    def get_data_dicts(pipeline):

        if pipeline == 'All':
            models = 'default'
        elif pipeline == 'Elastic-Net':
            models = ['elastic']
        elif pipeline == 'LGBM':
            models = ['lgbm']
        else:
            models = ['svm']

        r_df = get_ranks_sizes(results=results,
                               keep_full_name=True, threshold=False,
                               log=log, log_raw=log_raw,
                               add_ranks_labels=True,
                               add_raw=True, models=models, **parcels)

        data_dicts = []
        for name, df in r_df.groupby('Parcellation_Type'):
            data_dict = {
                'x': df['Size'],
                'y': df[metric_type],
                'mode': "markers",
                'text': df['full_name'],
                "marker": {
                    "size": marker_size,
                    "opacity": .5
                },
                "customdata": np.stack((df['r2'],
                                        df['roc_auc'],
                                        df['rank_label'],
                                        df['size_label']), axis=-1),
                'name': name,
                'hovertemplate': '<b>%{text}</b><br>' +
                                 '%{customdata[2]}<br>' +
                                 '%{customdata[3]}<br>' +
                                 'Mean R2: %{customdata[0]:.4f}<br>' +
                                 'Mean ROC AUC: %{customdata[1]:.4f}<br>' +
                                 '<extra></extra>'
            }

            data_dicts.append(data_dict)

        return data_dicts

    # make init data
    fig_dict["data"] = get_data_dicts(pipeline='LGBM')

    # make frames
    for model in models:
        frame = {"data": [], "name": str(model)}

        frame['data'] = get_data_dicts(model)
        fig_dict["frames"].append(frame)

        slider_step = {"args": [
            [model],
            {"frame": {"duration": 1000, "redraw": False},
             "transition": {"duration": 1000}}
        ],
            "label": model,
            "method": "animate"}

        sliders_dict["steps"].append(slider_step)

    fig_dict["layout"]["sliders"] = [sliders_dict]
    fig = go.Figure(fig_dict)
    
    return fig

def proc_metric_type(metric_type):
        
    if metric_type == 'r2':
        return 'Mean R2', True
    
    elif metric_type == 'roc_auc':
        return 'Mean ROC AUC', True

    return 'Mean Rank', False

## Base Single Parcellations -  log toggle

In [4]:
def gen_single_parcel_fig(metric_type='Mean_Rank', marker_size=20):
    
    # Set by metric type
    ylabel, log_raw = proc_metric_type(metric_type)
    
    # Init
    fig = init_fig(ylabel)

    def add_traces(r_df, visible=False):
        for name, df in r_df.groupby('Parcellation_Type'):
            add_go_base_scatter(fig, name, df, visible, y=metric_type, marker_size=marker_size)

    # Keep track of buttons
    buttons = []

    # Base
    add_traces(get_ranks_sizes(**params, add_raw=True,
                               log=False, log_raw=False,
                               add_ranks_labels=True), visible=True)
    buttons.append(dict(args=[{"visible": ts + fs}],
                        label='Base',
                        method="update"))

    # Log10
    add_traces(get_ranks_sizes(**params, add_raw=True,
                               log=True, log_raw=log_raw,
                               add_ranks_labels=True), visible=False)
    buttons.append(dict(args=[{"visible": fs + ts}],
                        label='Log',
                        method="update"))

    # Add base side dropdown + annot
    add_base_side_annot(fig, buttons)
    
    return fig

#plotly.offline.plot(gen_single_parcel_fig('Mean_Rank', marker_size=20))
gen_single_parcel_fig('Mean_Rank', marker_size=20).write_html('../docs/interactive1.html')
gen_single_parcel_fig('r2', marker_size=20).write_html('../docs/interactive1_r2.html')
gen_single_parcel_fig('roc_auc', marker_size=20).write_html('../docs/interactive1_roc_auc.html')

## Base Single Parcellations - by pipeline

In [5]:
def gen_pipeline_fig(metric_type='Mean_Rank', marker_size=20, log=True):
    
    if log:
        
        if metric_type == 'Mean_Rank':
            yrange = (1.25, 2.5)
        elif metric_type == 'r2':
            yrange = (-1.8, -.9)
        elif metric_type == 'roc_auc':
            yrange = (-.28, -.18)
            
        xrange = (.5, 4)

    else:
        
        if metric_type == 'Mean_Rank':
            yrange = (1, 230)
        elif metric_type == 'r2':
            yrange = (0, .15)
        elif metric_type == 'roc_auc':
            yrange = (.52, .66)
            
        xrange = (-250, 6250)
    
    return by_pipeline_base(metric_type,
                            marker_size=marker_size,
                            yrange=yrange,
                            xrange=xrange,
                            log=log,
                            **static)

def save_all(gen_func, base_num):
    
    for metric in ['Mean_Rank', 'r2', 'roc_auc']:
        for log in [True, False]:
            for marker_size in [10, 20]:
                
                # Gen figure
                fig = gen_func(metric, log=log, marker_size=marker_size)
                
                # Get different save tags
                base_tag = ''
                if not log:
                    base_tag = '_base'
                    
                metric_tag = ''
                if metric != 'Mean_Rank':
                    metric_tag = f'_{metric}'
                    
                includes_tag = ''
                if marker_size == 10:
                    includes_tag = '_includes/'
                
                # Save
                save_loc = f'../docs/{includes_tag}interactive{base_num}{base_tag}{metric_tag}.html'
                fig.write_html(save_loc)
                print('saved:', save_loc)


#plotly.offline.plot(gen_pipeline_fig('Mean_Rank', marker_size=20, log=False))
#plotly.offline.plot(gen_pipeline_fig('r2', marker_size=20, log=False))
#plotly.offline.plot(gen_pipeline_fig('roc_auc', marker_size=20, log=False))

save_all(gen_pipeline_fig, '2')

saved: ../docs/_includes/interactive2.html
saved: ../docs/interactive2.html
saved: ../docs/_includes/interactive2_base.html
saved: ../docs/interactive2_base.html
saved: ../docs/_includes/interactive2_r2.html
saved: ../docs/interactive2_r2.html
saved: ../docs/_includes/interactive2_base_r2.html
saved: ../docs/interactive2_base_r2.html
saved: ../docs/_includes/interactive2_roc_auc.html
saved: ../docs/interactive2_roc_auc.html
saved: ../docs/_includes/interactive2_base_roc_auc.html
saved: ../docs/interactive2_base_roc_auc.html


## By Target - avg over pipeline

In [6]:
def make_target_fig(marker_size=15, metric='Mean_Rank', **parcels):
    
    if metric == 'Mean_Rank':
        
        ylabel = 'Mean Rank'
        binary_only = False
        regression_only = False
        y = 'Mean_Rank'
        n = 45
        
    elif metric == 'r2':
        
        ylabel = 'Mean R2'
        binary_only = False
        regression_only = True
        y = 'Mean_Score'
        n = 22

    elif metric == 'roc_auc':
        
        ylabel = 'Mean ROC AUC'
        binary_only = True
        regression_only = False
        y = 'Mean_Score'
        n = 23

    fig = init_fig(ylabel=ylabel)
    
    def add_traces(r_df, y, visible=False):
        
        # Fixed colors
        palette = cycle(px.colors.qualitative.D3)

        # By target here
        for name, df in r_df.groupby('target'):
            add_go_by_target_scatter(fig, name, df, visible,
                                     palette, y=y,
                                     marker_size=marker_size)

    # Keep track of buttons
    buttons = []
    
    # Params to pass to get rank sizes
    params = {'results': results,
              'keep_full_name': True,
              'threshold': False,
              'avg_targets': False,
              'add_ranks_labels': True,
              'add_raw': False,
              'binary_only': binary_only,
              'regression_only': regression_only,
              **parcels}

    # Base
    add_traces(get_ranks_sizes(log=False, **params), y=y, visible=True)
    buttons.append(dict(args=[{"visible": ([True]*n) + ([False]*n)}],
                        label='Base', method="update"))

    # Log10
    add_traces(get_ranks_sizes(log=True, log_raw=True, **params), y=y, visible=False)
    buttons.append(dict(args=[{"visible": ([False]*n) + ([True]*n)}],
                        label='Log', method="update"))

    # Add base side dropdown + annot
    add_base_side_annot(fig, buttons)
    
    return fig


#plotly.offline.plot(make_target_fig(marker_size=15, metric='Mean_Rank', **static))
#plotly.offline.plot(make_target_fig(marker_size=15, metric='r2', **static))
#plotly.offline.plot(make_target_fig(marker_size=15, metric='roc_auc', **static))

make_target_fig(marker_size=15, metric='Mean_Rank', **static).write_html('../docs/interactive3.html')
make_target_fig(marker_size=15, metric='r2', **static).write_html('../docs/interactive3_r2.html')
make_target_fig(marker_size=15, metric='roc_auc', **static).write_html('../docs/interactive3_roc_auc.html')

## All - base w/ log toggle

In [7]:
def gen_multi_parcel_fig(metric_type='Mean_Rank', marker_size=20):
    
    # Set by metric type
    ylabel, log_raw = proc_metric_type(metric_type)
    
    # Init
    fig = init_fig(ylabel)

    def add_traces(r_df, visible=False):
        for name, df in r_df.groupby('Parcellation_Type'):
            add_go_base_scatter(fig, name, df, visible, y=metric_type, marker_size=marker_size)

    # Keep track of buttons
    buttons = []
    
    # Base
    r_df = get_single_vs_multiple_df(log=False, add_raw=True, add_ranks_labels=True,
                                     stacked=True, voted=True, grid=True,
                                     **params).reset_index()
    add_traces(r_df, visible=True)
    
    buttons.append(dict(args=[{"visible": ts + fs}],
                        label='Base',
                        method="update"))

    # Log10
    r_df = get_single_vs_multiple_df(log=True, log_raw=log_raw, add_raw=True,
                                     add_ranks_labels=True,
                                     stacked=True, voted=True, grid=True,
                                     **params).reset_index()
    add_traces(r_df, visible=False)

    buttons.append(dict(args=[{"visible": fs + ts}],
                        label='Log',
                        method="update"))

    # Add base side dropdown + annot
    add_base_side_annot(fig, buttons)
    
    return fig

#plotly.offline.plot(gen_multi_parcel_fig(metric_type='roc_auc', marker_size=20))

gen_multi_parcel_fig(marker_size=20).write_html('../docs/interactive4.html')
gen_multi_parcel_fig(metric_type='r2', marker_size=20).write_html('../docs/interactive4_r2.html')
gen_multi_parcel_fig(metric_type='roc_auc', marker_size=20).write_html('../docs/interactive4_roc_auc.html')

## All - by pipeline

In [8]:
def gen_all_pipeline_fig(metric_type='Mean_Rank',
                         marker_size=20, log=True):
    
    if log:
        
        if metric_type == 'Mean_Rank':
            yrange = (1.1, 2.9)
        elif metric_type == 'r2':
            yrange = (-1.9, -.8)
        elif metric_type == 'roc_auc':
            yrange = (-.30, -.16)
            
        xrange = (.6, 4)

    else:
        
        if metric_type == 'Mean_Rank':
            yrange = (1, 450)
        elif metric_type == 'r2':
            yrange = (0, .12)
        elif metric_type == 'roc_auc':
            yrange = (.52, .66)
            
        xrange = (-250, 8000)
    
    return by_pipeline_base(metric_type,
                            marker_size=marker_size,
                            yrange=yrange,
                            xrange=xrange,
                            log=log,
                            **all_parcels)

#plotly.offline.plot(gen_all_pipeline_fig(metric_type='Mean_Rank', marker_size=20, log=True))
#plotly.offline.plot(gen_all_pipeline_fig(metric_type='Mean_Rank', marker_size=20, log=False))

#plotly.offline.plot(gen_all_pipeline_fig(metric_type='r2', marker_size=20, log=True))
#plotly.offline.plot(gen_all_pipeline_fig(metric_type='r2', marker_size=20, log=False))

#plotly.offline.plot(gen_all_pipeline_fig(metric_type='roc_auc', marker_size=20, log=True))
#plotly.offline.plot(gen_all_pipeline_fig(metric_type='roc_auc', marker_size=20, log=False))

save_all(gen_func=gen_all_pipeline_fig, base_num='5')

saved: ../docs/_includes/interactive5.html
saved: ../docs/interactive5.html
saved: ../docs/_includes/interactive5_base.html
saved: ../docs/interactive5_base.html
saved: ../docs/_includes/interactive5_r2.html
saved: ../docs/interactive5_r2.html
saved: ../docs/_includes/interactive5_base_r2.html
saved: ../docs/interactive5_base_r2.html
saved: ../docs/_includes/interactive5_roc_auc.html
saved: ../docs/interactive5_roc_auc.html
saved: ../docs/_includes/interactive5_base_roc_auc.html
saved: ../docs/interactive5_base_roc_auc.html


## Ensemble - by pipeline

In [9]:
def gen_ensemble_only_pipeline_fig(metric_type='Mean_Rank',
                                   marker_size=20, log=True):
    
    if log:

        if metric_type == 'Mean_Rank':
            yrange = (1.2, 2.4)
        elif metric_type == 'r2':
            yrange = (-1.6, -.8)
        elif metric_type == 'roc_auc':
            yrange = (-.27, -.16)
            
        xrange = (2.3, 4)

    else:
        
        if metric_type == 'Mean_Rank':
            yrange = (1, 125)
        elif metric_type == 'r2':
            yrange = (.04, .12)
        elif metric_type == 'roc_auc':
            yrange = (.56, .66)
            
        xrange = (300, 8000)
    
    return by_pipeline_base(metric_type,
                            marker_size=marker_size,
                            yrange=yrange,
                            xrange=xrange,
                            log=log,
                            **ensemble_only)

#plotly.offline.plot(gen_ensemble_only_pipeline_fig(metric_type='Mean_Rank', marker_size=20, log=True))
#plotly.offline.plot(gen_ensemble_only_pipeline_fig(metric_type='Mean_Rank', marker_size=20, log=False))

#plotly.offline.plot(gen_ensemble_only_pipeline_fig(metric_type='r2', marker_size=20, log=True))
#plotly.offline.plot(gen_ensemble_only_pipeline_fig(metric_type='r2', marker_size=20, log=False))

#plotly.offline.plot(gen_ensemble_only_pipeline_fig(metric_type='roc_auc', marker_size=20, log=True))
#plotly.offline.plot(gen_ensemble_only_pipeline_fig(metric_type='roc_auc', marker_size=20, log=False))

save_all(gen_func=gen_ensemble_only_pipeline_fig, base_num='6')

saved: ../docs/_includes/interactive6.html
saved: ../docs/interactive6.html
saved: ../docs/_includes/interactive6_base.html
saved: ../docs/interactive6_base.html
saved: ../docs/_includes/interactive6_r2.html
saved: ../docs/interactive6_r2.html
saved: ../docs/_includes/interactive6_base_r2.html
saved: ../docs/interactive6_base_r2.html
saved: ../docs/_includes/interactive6_roc_auc.html
saved: ../docs/interactive6_roc_auc.html
saved: ../docs/_includes/interactive6_base_roc_auc.html
saved: ../docs/interactive6_base_roc_auc.html


## All By Target - avg over pipeline

In [10]:
#plotly.offline.plot(make_target_fig(metric='Mean_Rank', marker_size=15, **all_parcels))
#plotly.offline.plot(make_target_fig(metric='r2', marker_size=15, **all_parcels))
#plotly.offline.plot(make_target_fig(metric='roc_auc', marker_size=15, **all_parcels))

make_target_fig(metric='Mean_Rank', marker_size=15, **all_parcels).write_html('../docs/interactive7.html')
make_target_fig(metric='r2', marker_size=15, **all_parcels).write_html('../docs/interactive7_r2.html')
make_target_fig(metric='roc_auc', marker_size=15, **all_parcels).write_html('../docs/interactive7_roc_auc.html')